In [1]:
### import necessary libraries ###

import os
import random
import itertools
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

In [2]:
### Defining parameters  ###
SHAPE = (224,224,3)
batch_size = 256

In [3]:
### Generator Wrapper to Create Fake Label ###

def wrap_generator(generator):
    
    while True:
        x,y = next(generator)
        y = tf.keras.utils.to_categorical(y)
        zeros = tf.zeros_like(y) + tf.constant([1.,0.])
        y = tf.concat([y,zeros], axis=0)
        
        yield x,y

In [4]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def get_model(train=True):
    
    set_seed(33)
    ### pre-trained VGG model ###
    pre_process = Lambda(preprocess_input)
    vgg = VGG16(weights = 'imagenet', include_top = True, input_shape = SHAPE)
    vgg = Model(vgg.input, vgg.layers[-3].output)
    vgg.trainable = False
    
    inp = Input(SHAPE)
    vgg_16_process = pre_process(GaussianNoise(0.1)(inp))
    vgg_out = vgg(vgg_16_process)
    
    noise = Lambda(tf.zeros_like)(vgg_out)
    noise = GaussianNoise(0.1)(noise)

    if train:
        x = Lambda(lambda z: tf.concat(z, axis=0))([vgg_out,noise])
        x = Activation('relu')(x)
    else:
        x = vgg_out
        
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inp, out)
    ### Compile the model ###
    model.compile(Adam(lr=1e-4), loss='binary_crossentropy')
    return model

In [5]:
### Create Empty Generators ###

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [6]:
### Flow Generators ###

train_generator = train_datagen.flow_from_directory(
            '/Users/ab0707/Downloads/ai_coding_test-main/training',
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 33,
            classes = ['apples']
    )
### External test data was taken which were not apples ###
test_generator = test_datagen.flow_from_directory(
            '/Users/ab0707/Downloads/ai_coding_test-main/test',
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 33,
            classes = ['not apples','apples']
)

Found 8 images belonging to 1 classes.
Found 8 images belonging to 2 classes.


In [7]:
### Fitting the model to the images using Early stopping ###
es = EarlyStopping(monitor='val_loss', mode='auto', restore_best_weights=True, verbose=1, patience=5)

model = get_model()
model.fit(wrap_generator(train_generator), steps_per_epoch=np.ceil(train_generator.samples/train_generator.batch_size), epochs=100)

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 6.7991
Epoch 2/100
1/1 [==============================] - 1s 614ms/step - loss: 3.8173
Epoch 3/100
1/1 [==============================] - 1s 666ms/step - loss: 1.6331
Epoch 4/100
1/1 [==============================] - 1s 605ms/step - loss: 0.3775
Epoch 5/100
1/1 [==============================] - 1s 600ms/step - loss: 0.3179
Epoch 6/100
1/1 [==============================] - 1s 596ms/step - loss: 0.3073
Epoch 7/100
1/1 [==============================] - 1s 597ms/step - loss: 0.3017
Epoch 8/100
1/1 [==============================] - 1s 594ms/step - loss: 0.2964
Epoch 9/100
1/1 [==============================] - 1s 594ms/step - loss: 0.2863
Epoch 10/100
1/1 [==============================] - 1s 610ms/step - loss: 0.2808
Epoch 11/100
1/1 [==============================] - 1s 606ms/step - loss: 0.2652
Epoch 12/100
1/1 [==============================] - 1s 606ms/step - loss: 0.2658
Epoch 13/100
1/1 [======================

In [8]:
### Retrive Test Label from Generator ###

test_num = test_generator.samples

label_test = []
for i in range((test_num // test_generator.batch_size)+1):
    X,y = test_generator.next()
    label_test.append(y)
        
label_test = np.argmax(np.vstack(label_test), axis=1)
label_test.shape

(8,)

In [9]:
### Switch to Infernece mode to compute predictions ###

inference_model = get_model(train=False)
inference_model.set_weights(model.get_weights())

In [10]:
### Compute Predictions on Test Data ###

pred_test = np.argmax(inference_model.predict(test_generator), axis=1)

In [11]:
# Making new predictions testing model on test data ###

from keras.preprocessing import image
test_image = image.load_img('/Users/ab0707/Downloads/ai_coding_test-main/test/not apples/00001.png', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = inference_model.predict(test_image)
train_generator.class_indices
if result[0][0] == 1:
    prediction = 'not apples'
    print(prediction)
else:
    prediction = 'apples'
    print(prediction)

not apples


In [ ]:
### Here external test data was taken which were not apples and then model was ###
### trained on apples data and test on not apples data ###